# Automação Web e Busca de Informações com Python

__Desafio:__
Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a contação destes 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso vamos criar uma automação Web:
- Usaremos o Selenium.
- Importante: baixar o webdriver - https://chromedriver.chromium.org/downloads (inserir arquivo no path do Anaconda)

In [1]:
# !pip install selenium
# Chrome -> chromedriver
# Firefox -> geckodriver
    # Após baixado, colocar na pasta onde está instalado o python (Caminho mostrado no anaconda)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Passo 1: Pegar a cotação do dólar
    # Abrir o navegador
navegador = webdriver.Chrome()

    # Entrar no google
navegador.get("https://www.google.com.br/")

    # Pesquisar a cotação do dólar no google
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    # Pegar a cotação
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# Passo 2: Pegar a cotação do euro
navegador = webdriver.Chrome()
navegador.get("https://www.google.com.br/")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador = webdriver.Chrome()
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

navegador.quit()

5.1728
5.2865
293.37


In [3]:
# Passo 4: Atualizar a base de dados
import pandas as pd

tabela = pd.read_excel("22-07-27/Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [4]:
# Passo 5: Recalcular os preços
    # Atualizar as cotações
tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro", "Cotação"] = float(cotacao_ouro)
    
    # Preço de compra: Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
    
    # Preço de venda: Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1728,5172.748272,1.40,7241.847581
1,Carro Renault,4500.00,Euro,5.2865,23789.250000,2.00,47578.500000
2,Notebook Dell,899.99,Dólar,5.1728,4655.468272,1.70,7914.296062
3,IPhone,799.00,Dólar,5.1728,4133.067200,1.70,7026.214240
4,Carro Fiat,3000.00,Euro,5.2865,15859.500000,1.90,30133.050000
5,Celular Xiaomi,480.48,Dólar,5.1728,2485.426944,2.00,4970.853888
6,Joia 20g,20.00,Ouro,293.3700,5867.400000,1.15,6747.510000


In [5]:
# Passo 6: Exportar a base de dados
tabela.to_excel("22-07-27/Produtos Novo.xlsx", index=False)